```
RIM 城镇人均犯罪率
ZN 占地面积超过 25,000 平方英尺的住宅用地比例
INDUS 每个城镇非零售业务的比例
CHAS Charles River 虚拟变量(如果是河道，则为 1；否则为 0)
NOX 一氧化氮浓度(每千万份)
RM 每间住宅的平均房间数
AGE 1940 年以前建造的自住单位比例
DIS 加权距离波士顿的五个就业中心
RAD 径向高速公路的可达性指数
TAX 每 10,000 美元的全额物业税率
PTRATIO 城镇的学生与教师比例
B 1000 (Bk-0.63)^2 其中 Bk 是城镇黑人的比例
LSTAT 人口状况下降%
MEDV 自有住房的中位数报价，单位 1000 美元
```

In [38]:
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import seaborn as sns

In [7]:
boston = datasets.load_boston()

In [465]:
from sklearn import datasets
y = boston['target']
X = pd.DataFrame(data=boston['data'], columns=boston['feature_names'])
X[:3]

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03


In [424]:
def principal_component_analysis(X, contrib_threshold=0.75):
    '''主成分分析'''
    n_principals = -1
    
    # 将样本标准化处理
    X_norm = (X - X.mean()) / X.std()
    # 计算样本协方差矩阵
    X_corr = (X_norm.T @ X_norm) / (X_norm.shape[0]-1)
    # 协方差矩阵的特征值分解
    feat_value, feat_vec = np.linalg.eig(X_corr)
    
    # 选取主成分个数：取前6个特征，可使累积方差贡献率超过 85%
    sum_feat_value = sum(feat_value)
    arg_index = np.argsort(featValue)
    accum_contrib = []
    accum = 0
    for i in range(-1,-14,-1):
        accum += featValue[arg_index[i]]
        accum_contrib.append(accum / sum_feat_value)
        if n_principals == -1 and accum / sum_feat_value >= contrib_threshold:
            n_principals = -i
            print('选取主成分个数={} 累积方差贡献率={}'.format(
                   n_principals, accum / sum_feat_value))
    
    # 样本进行正交变换
    M = pd.DataFrame(feat_vec)
    X_trans = M @ (X_norm.to_numpy().T)
    return n_principals, accum_contrib, arg_index, X_trans.T, M
n_principals, accum_contrib, arg_index, X_trans, M = principal_component_analysis(
            X, contrib_threshold=0.85)

选取主成分个数=6 累积方差贡献率=0.8578876032275043


In [423]:
def linear_regression(X_train, y_train):
    '''使用最小二乘法建立线性回归模型'''
    linreg = LinearRegression()
    linreg.fit(X_train, y_train)
    print('估计独立项：', linreg.intercept_)
    print('估计系数', linreg.coef_)
    return linreg

def linear_regression2(X_train, y_train):
    X_train = X_train.copy()
    X_train.insert(0, 'ones', 1)
    beta = np.linalg.inv(X_train.T @ X_train) @ X_train.T @ y_train
    return beta

In [425]:
for contrib_threshold in (0.85,):
    n_principals, accum_contrib, arg_index, X_trans, M = principal_component_analysis(X, contrib_threshold)
    X_train, X_valid, y_train, y_valid = train_test_split(X_trans,y,
                                                      test_size=0.25,
                                                      random_state=33)
    X_train_slice = X_train.iloc[:, arg_index[-1:-n_principals-1:-1]]
    linreg = linear_regression(X_train_slice, y_train)
    print('手算的估计系数：', list(linear_regression2(X_train_slice, y_train)))
    print('训练集上的拟合优度：', linreg.score(X_train.iloc[:, arg_index[-1:-n_principals-1:-1]], y_train))
    print('测试集上的拟合优度：', linreg.score(X_valid.iloc[:, arg_index[-1:-n_principals-1:-1]], y_valid))
    print()

选取主成分个数=6 累积方差贡献率=0.8578876032275043
估计独立项： 22.62325960685469
估计系数 [ 3.78196814  3.70971566  2.10593225 -2.65573611  1.06234712 -3.64574835]
手算的估计系数： [22.6232596068547, 3.781968141245958, 3.709715662293321, 2.105932252170091, -2.65573610909668, 1.0623471199193601, -3.645748350718738]
训练集上的拟合优度： 0.5856598462994655
测试集上的拟合优度： 0.4923510891595909



In [438]:
X_extend = X.copy()
X_extend.insert(0,'Const',1)
est = sm.OLS(y, X_extend)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     108.1
Date:                Mon, 28 Dec 2020   Prob (F-statistic):          6.72e-135
Time:                        13:44:56   Log-Likelihood:                -1498.8
No. Observations:                 506   AIC:                             3026.
Df Residuals:                     492   BIC:                             3085.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Const         36.4595      5.103      7.144      0.0

In [455]:
X_extend2 = X.copy()
X_extend2.insert(0,'Const',1)
X_extend2 = X_extend2.drop(['INDUS', 'AGE', 'CHAS', 'CRIM', 'ZN', 'TAX', 'RAD'], axis=1)
est = sm.OLS(y, X_extend2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.715
Model:                            OLS   Adj. R-squared:                  0.712
Method:                 Least Squares   F-statistic:                     209.0
Date:                Mon, 28 Dec 2020   Prob (F-statistic):          1.11e-132
Time:                        13:56:40   Log-Likelihood:                -1522.3
No. Observations:                 506   AIC:                             3059.
Df Residuals:                     499   BIC:                             3088.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Const         30.5170      4.960      6.153      0.0